# 05 -- Diagonal running speed adjustment factor
April 23, 2024

## Problem statement
In coord system like Pygame's, moving an object diagonally (45°, 135°, 225°, or 315°) will be 40% faster than horizontally or vertically (0°, 90°, 180° or 270°). Till now, I've adjusted pure diagonal movement by 0.744.

Now, I want to be able to scale this for movement that is not exactly 45°. So, a movement at 5° -- about 11% of 45° -- should re-scale the diagonal adjustment from 0.744 (74.4%) to 0.083 (8.3%). And a movement at 40° should give me a re-scaled diagonal adjustment of 0.661 (66.1%).

### A few problems to solve
For now, let's only look at diagonal movements between 0° and 180°... my current trigonometry foundation in sandbox --> Pygame_trig_1.py doesn't account for 180° to 360° (South east to South west).

1. I need a function that converts a compass direction to an absolute distance on the number line from the nearest multiple of 45°. 
2. Then I need to scale my diagonal adjustment factor by that.
3. Then I need to adjust my speed by that factor.

In [1]:
import math

### 0. What is the exact 100% diagonal adjustment factor
- From $1^2 + 1^2 = hyp^2$ > sqrt(1+1)  

In [51]:
man_diagonal_factor = 1/math.sqrt(2)
man_diagonal_factor = 1 - man_diagonal_factor
man_diagonal_factor

0.29289321881345254

### 0. What is the exact speed?

In [10]:
man_speed = 4/3
man_speed

1.3333333333333333

In [49]:
man_speed * man_diagonal_factor

0.9428090415820632

### 0. Build helper func to test the main func 
- 'steps' = how granular to look. 15 is optimal.

In [73]:
def unit_test(steps):
    for deg in range(0, 180, steps):
        factor = get_speed_adjustment_factor(deg)

### 1. Convert compass heading to an absolute distance from a multiple of 45 (0° to 180° for now)

### 2. Scale my diagonal adjustment factor by that

### 3. Adjust my speed by that factor

In [59]:
def get_speed_adjustment_factor(deg):
    if deg > 90:
        deg -= 90
    
    
    """
    What if I adjust diagonal factor to be 1-0.71, or 0.2929
    
    Given 45°, I want to adjust speed by 100% of the diagonal_factor to maximize speed reduction:
        - scaled_diagonal_factor * 100% = 0.29 * 1.00 = 0.29
        - new_speed = old_speed - (old speed * scaled_diagonal_factor) = 1.33 - (1.33 * 0.29) = 0.94
    Now, given 20°, intuitively, I want to adjust speed less -- by only 44% of the diagonal factor
        - scaled_diagonal_factor * 44% = 0.71 * 0.44 = 0.13
        - new_speed = old_speed * (1 - scaled_diagonal_factor) = 1.33 - (0.87) = 1.16
    """
    
    inverse_distance_from_45 = 45 - abs(deg-45)
    percent_of_45 = inverse_distance_from_45/45
    

    
    scaled_diagonal_factor = man_diagonal_factor * percent_of_45
    
    scaled_speed = man_speed * (1 - scaled_diagonal_factor)
    
    
    
    
    ## Rounding
    percent_of_45 = round(percent_of_45, 2)
    scaled_diagonal_factor = round(scaled_diagonal_factor, 3)
    scaled_speed = round(scaled_speed, 2)

    
    ## Print results
    print(f"Deg: {deg}  |  Inverse distance from 45: {inverse_distance_from_45}", end = "  |  ")
    print(f"Percent of 45: {percent_of_45}  | scaled_diagonal_factor: {scaled_diagonal_factor}", end = "  |  ") 
    print(f"Scaled_speed: {scaled_speed} ")


unit_test(15)

Deg: 0  |  Inverse distance from 45: 0  |  Percent of 45: 0.0  | scaled_diagonal_factor: 0.0  |  Scaled_speed: 1.33 
Deg: 15  |  Inverse distance from 45: 15  |  Percent of 45: 0.33  | scaled_diagonal_factor: 0.098  |  Scaled_speed: 1.2 
Deg: 30  |  Inverse distance from 45: 30  |  Percent of 45: 0.67  | scaled_diagonal_factor: 0.195  |  Scaled_speed: 1.07 
Deg: 45  |  Inverse distance from 45: 45  |  Percent of 45: 1.0  | scaled_diagonal_factor: 0.293  |  Scaled_speed: 0.94 
Deg: 60  |  Inverse distance from 45: 30  |  Percent of 45: 0.67  | scaled_diagonal_factor: 0.195  |  Scaled_speed: 1.07 
Deg: 75  |  Inverse distance from 45: 15  |  Percent of 45: 0.33  | scaled_diagonal_factor: 0.098  |  Scaled_speed: 1.2 
Deg: 90  |  Inverse distance from 45: 0  |  Percent of 45: 0.0  | scaled_diagonal_factor: 0.0  |  Scaled_speed: 1.33 
Deg: 15  |  Inverse distance from 45: 15  |  Percent of 45: 0.33  | scaled_diagonal_factor: 0.098  |  Scaled_speed: 1.2 
Deg: 30  |  Inverse distance from 45:

# ^^^ Yup -- that works
- At 0° and 90°, the scaled speed = full speed > 1.33
- At 45°, the scaled speed is adjusted by 100% of the adjustment factor = 1.33 - 0.29 = 0.94
- At 15° and 75°, the scaled speed is adjusted by only 1/3 of the adjustment factor = 1.33 - 0.1 = 1.23

## Clean version of the code above

In [65]:
def get_scaled_speed(deg):
    if deg > 90:
        deg -= 90
    
    ## Get value to scale the diagonal factor by
    inverse_distance_from_45 = 45 - abs(deg-45)
    percent_of_45 = inverse_distance_from_45/45
    
    ## Scale the diagonal factor
    scaled_diagonal_factor = man_diagonal_factor * percent_of_45
    
    ## Scale the speed
    scaled_speed = man_speed * (1 - scaled_diagonal_factor)
    
    return scaled_speed

get_speed_adjustment_factor(15)

1.2031585694162432

## Let's incorporate movement South
- Reduce all degress to a number from 0-90

In [71]:
def reduce_to_under_90(deg):
    factor = deg//90
    deg -= 90*factor
    
    return deg
    
for i in range(0, 361, 15):
    x = reduce_to_under_90(i)    
    print(i, "  |  ", x)

    #deg = 200

0   |   0
15   |   15
30   |   30
45   |   45
60   |   60
75   |   75
90   |   0
105   |   15
120   |   30
135   |   45
150   |   60
165   |   75
180   |   0
195   |   15
210   |   30
225   |   45
240   |   60
255   |   75
270   |   0
285   |   15
300   |   30
315   |   45
330   |   60
345   |   75
360   |   0


In [81]:
def get_scaled_speed(deg):
    factor = deg//90
    deg -= 90*factor
    
    ## Get value to scale the diagonal factor by
    inverse_distance_from_45 = 45 - abs(deg-45)
    percent_of_45 = inverse_distance_from_45/45
    
    ## Scale the diagonal factor
    scaled_diagonal_factor = man_diagonal_factor * percent_of_45
    
    ## Scale the speed
    scaled_speed = man_speed * (1 - scaled_diagonal_factor)
    
    return scaled_speed, deg

get_scaled_speed(15)

(1.2031585694162432, 15)

In [82]:
def unit_test2(steps):
    for deg in range(0, 361, steps):
        scaled_speed, adjusted_deg = get_scaled_speed(deg)
        print(adjusted_deg, "  |  ", round(scaled_speed, 3))

unit_test2(15)

0   |   1.333
15   |   1.203
30   |   1.073
45   |   0.943
60   |   1.073
75   |   1.203
0   |   1.333
15   |   1.203
30   |   1.073
45   |   0.943
60   |   1.073
75   |   1.203
0   |   1.333
15   |   1.203
30   |   1.073
45   |   0.943
60   |   1.073
75   |   1.203
0   |   1.333
15   |   1.203
30   |   1.073
45   |   0.943
60   |   1.073
75   |   1.203
0   |   1.333


# New problem, convert theta to a direction, Left, Right, other (North/South)

In [9]:
clockwise_thetas = []
for theta in range(0, -179, -15):
    clockwise_thetas.append(theta)

for theta in range(180, 0, -15):
    clockwise_thetas.append(theta)

print(clockwise_thetas)

[0, -15, -30, -45, -60, -75, -90, -105, -120, -135, -150, -165, 180, 165, 150, 135, 120, 105, 90, 75, 60, 45, 30, 15]


- 0 = right
- 90 = North
- 180 = left
- -135 = SW
- -90 = South
- -45 = SE

In [43]:
def get_direction_facing(theta_deg):
    
    ## Set a constant in deg that says how close I need to be to 90 and 180 for the man to not face North
    left_right_threshold = 60
    
    distance_from_left = abs( 180 - abs(theta_deg) )
    distance_from_right = abs( 0 - abs(theta_deg) )
    
    #print(f"Deg = {theta_deg}  |  Distance from left = {distance_from_left}  |  Distance from right = {distance_from_right} ")
    #return

    if distance_from_left < left_right_threshold:
        print(f"  > Deg: {theta_deg}  |  Left, with abs distance from 180 of {distance_from_left}")
        return 1 # 1 = left
        
    if distance_from_right < left_right_threshold:
        print(f"  > Deg: {theta_deg}  |  Right, with abs distance from 0 of {distance_from_right}")
        return 2 # 2 = right
    
    print(f"  > Deg: {theta_deg}  |  North/South")
    return 3

In [47]:
def unit_test3():
    for theta in clockwise_thetas:
        direction = get_direction_facing(theta)

        text = "North"

        if direction == 1:
            text = "Left"

        elif direction == 2:
            text = "Right"

        #print(f"Theta: {theta}  |  Direction: {text} ")
unit_test3()

  > Deg: 0  |  Right, with abs distance from 0 of 0
  > Deg: -15  |  Right, with abs distance from 0 of 15
  > Deg: -30  |  Right, with abs distance from 0 of 30
  > Deg: -45  |  Right, with abs distance from 0 of 45
  > Deg: -60  |  North/South
  > Deg: -75  |  North/South
  > Deg: -90  |  North/South
  > Deg: -105  |  North/South
  > Deg: -120  |  North/South
  > Deg: -135  |  Left, with abs distance from 180 of 45
  > Deg: -150  |  Left, with abs distance from 180 of 30
  > Deg: -165  |  Left, with abs distance from 180 of 15
  > Deg: 180  |  Left, with abs distance from 180 of 0
  > Deg: 165  |  Left, with abs distance from 180 of 15
  > Deg: 150  |  Left, with abs distance from 180 of 30
  > Deg: 135  |  Left, with abs distance from 180 of 45
  > Deg: 120  |  North/South
  > Deg: 105  |  North/South
  > Deg: 90  |  North/South
  > Deg: 75  |  North/South
  > Deg: 60  |  North/South
  > Deg: 45  |  Right, with abs distance from 0 of 45
  > Deg: 30  |  Right, with abs distance from 

## Okay -- that works. Need to clean up the code

In [54]:
def get_direction_facing(theta_deg):
    
    ## Set a constant in deg that says how close I need to be to 90 and 180 for the man to not face North
    left_right_threshold = 60
    
    distance_from_left = abs( 180 - abs(theta_deg) )
    distance_from_right = abs( 0 - abs(theta_deg) )
    
    if distance_from_left < left_right_threshold:
        return 1 # 1 = left
        
    if distance_from_right < left_right_threshold:
        return 2 # 2 = right
    
    return 3 # = North/South



for theta in clockwise_thetas:
    print(theta, "  |  ", get_direction_facing(theta))

0   |   2
-15   |   2
-30   |   2
-45   |   2
-60   |   3
-75   |   3
-90   |   3
-105   |   3
-120   |   3
-135   |   1
-150   |   1
-165   |   1
180   |   1
165   |   1
150   |   1
135   |   1
120   |   3
105   |   3
90   |   3
75   |   3
60   |   3
45   |   2
30   |   2
15   |   2
